In [ ]:
#Manipolazione Dati
import geopandas as gpd
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from itertools import groupby
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
#Manipolazione HTML
from IPython.display import display, HTML, Markdown
import io
import base64

Mattia Nadal - 167372 <br>
Università degli Studi di Udine - A.A. 2024/2025
# Analisi sui dati dei vincitori del Pallone D'oro maschile 1956-2024

## Perchè analizzare i vincitori?
* L'interesse per il calcio
* Il desiderio di capire l'evoluzione che il trofeo ha avuto
* Quali sono le carattestiche più comuni dei vincitori

## I dataset:
* Datset iniziale reperibile su Kagle a questo [indirizzo](https://www.kaggle.com/datasets/seriadiallo1/ballon-dor):
  * Nome: ballon d'or
  * Creatore: Sadialiou Diallo
  * Anno: 2020
* Dataset ulteriori creati tramite web scraping dai seguenti siti:
  * [Wikipedia](https://it.wikipedia.org/wiki/Pallone_d%27oro)
  * [Transfermarkt](https://www.transfermarkt.it/)

### Dati rilevanti
* Generalità giocatore (Nome giocatore, Età, Nazionalità)
* Caratteristiche giocatore (Squadra di appartenenza, Ruolo)
* Posizionamento sul podio (Rank per anno)
* Competizioni vinte per anno (Campionato, Coppa nazionale, Campionati internazionali...)

## Domande
1. Quali nazioni hanno vinto più palloni d'oro?
2. Come sono disdribuiti sul podio i giocatori in rapporto alla nazionalità?
3. Con quali trofei si hanno più possibilità di vincere il premio?

# 1. Quali nazioni hanno vinto più palloni d'oro?
### Premessa
Nel corso del tempo, il metodo di votazione è cambiato più volte per riflettere l'evoluzione del calcio e aumentare l'equità e la rappresentatività del premio:
* 1956–1994: Giuria di Giornalisti Europei
* 1995-2009: Globalizzazione della Giuria
* 2010-2015: Fusione con il FIFA World Player
* 2016-oggi: Limitata ai giornalisti dei primi 100 paesi nella classifica FIFA maschile


In [ ]:
gold_gradient = LinearSegmentedColormap.from_list(
    "gold_gradient",
    ["#fff8dc", "#ffd700", "#b8860b"]  # Chiaro (cornsilk), oro, oro scuro
)

In [ ]:
display(Markdown("Visualizziamo quali nazioni tra il 1956 e il 1994 hanno vinto il premio"))

#Lettura file geojson
mappaEuropea = gpd.read_file("../Grafici/world_pre1995.geojson")

# Colori da usare per il gradiente oro
cmap = gold_gradient
# Filtra solo gli stati da colorare (escludi quelli bianchi)
mappa_da_colorare = mappaEuropea[mappaEuropea['color'].str.upper() != '#FFFFFF']
# Normalizzazione sui soli stati da colorare
norm = mcolors.Normalize(
    vmin=mappa_da_colorare['Numero'].min(),
    vmax=mappa_da_colorare['Numero'].max()
)
# Assegna nuovi colori (solo a quelli che non sono bianchi)
mappaEuropea['color'] = mappaEuropea.apply(
    lambda row: cmap(norm(row['Numero'])) if row['color'].upper() != '#FFFFFF' else 'whitesmoke',
    axis=1
)

# Crea la mappa con i colori specificati e bordi neri con una linea sottile
fig, ax = plt.subplots(1, 1, figsize=(5, 10))
mappaEuropea.boundary.plot(ax=ax, linewidth=0.1, color='black')  # Linea dei bordi sottile
mappaEuropea.plot(ax=ax, color=mappaEuropea['color'], edgecolor='black')

# Aggiungere la legenda manualmente
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, orientation='horizontal', fraction=0.036, pad=0.04)
cbar.set_label('Occorrenze vincitori per nazione', size = 15)  # <-- dimensione dell’etichetta
ax.set_title('Nazioni con giocatori vicenti fino al 1994', size = 15)
ax.set_facecolor('azure')  # Imposta il colore di sfondo a azzurrino chiaro
plt.tight_layout()  # Per togliere spazi bianchi inutili

# Mostra la mappa
# Salva la figura in un buffer
buf = io.BytesIO()
plt.savefig(buf, format='png', bbox_inches='tight')
buf.seek(0)
img_base64 = base64.b64encode(buf.read()).decode('utf-8')
buf.close()
plt.close(fig)

# Mostra immagine centrata
display(HTML(f"""
<div style="display: flex; justify-content: center;">
    <img src="data:image/png;base64,{img_base64}"/>
</div>
"""))

In [ ]:
display(Markdown("Visualizziamo quali nazioni dal 1995 ad oggi hanno vinto il premio"))

#Lettura file geojson
mappaMondo = gpd.read_file("../Grafici/world_post1995.geojson")

#Mappa colori
cmap = gold_gradient
# Filtra solo gli stati da colorare (escludi quelli bianchi)
mappa_da_colorare = mappaMondo[mappaMondo['color'].str.upper() != '#FFFFFF']
# Normalizzazione sui soli stati da colorare
norm = mcolors.Normalize(
    vmin=mappa_da_colorare['Numero'].min(),
    vmax=mappa_da_colorare['Numero'].max()
)
# Assegna nuovi colori (solo a quelli che non sono bianchi)
mappaMondo['color'] = mappaMondo.apply(
    lambda row: cmap(norm(row['Numero'])) if row['color'].upper() != '#FFFFFF' else 'whitesmoke',
    axis=1
)

# Inizializza figure
fig = go.Figure()

# Crea la mappa con i colori specificati e bordi neri con una linea sottile
fig, ax = plt.subplots(1, 1, figsize=(13, 12))
mappaMondo.boundary.plot(ax=ax, linewidth=0.1, color='black')  # Linea dei bordi sottile
mappaMondo.plot(ax=ax, color=mappaMondo['color'], edgecolor='black')

# Aggiungere la legenda manualmente
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, orientation='horizontal', fraction=0.04, pad=0.04)
# Imposta dimensione del testo della legenda
cbar.ax.tick_params(labelsize=10)  # <-- dimensione dei numeri
cbar.set_label('Occorrenze vincitori per nazione', size = 15)  # <-- dimensione dell’etichetta
ax.set_title('Nazioni con giocatori vicenti dal 1995 ad oggi', size = 20)
ax.set_facecolor('azure')  # Imposta il colore di sfondo a azzurrino chiaro

# Salva la figura in un buffer
buf = io.BytesIO()
plt.savefig(buf, format='png', bbox_inches='tight')
buf.seek(0)
img_base64 = base64.b64encode(buf.read()).decode('utf-8')
buf.close()
plt.close(fig)

# Mostra immagine centrata
display(HTML(f"""
<div style="display: flex; justify-content: center;">
    <img src="data:image/png;base64,{img_base64}"/>
</div>
"""))

In [ ]:
import plotly.express as px

df = px.data.election()
geojson = px.data.election_geojson()

fig = px.choropleth(df, geojson=geojson, color="winner",
                    locations="district", featureidkey="properties.district",
                    projection="mercator", hover_data=["Bergeron", "Coderre", "Joly"]
                   )

# 2. Come sono disdribuiti sul podio i giocatori in rapporto alla nazionalità?


In [ ]:
display(Markdown("Visualizziamo una Heapmap che confronta i ruoli dei giocatori in rapporto alla nazionalità"))

df = pd.read_csv('../Dataset/dfHeapmap.csv')

# --- Funzioni per etichette raggruppate ---
def add_line(ax, xpos, ypos):
    line = plt.Line2D([ypos, ypos + 0.2], [xpos, xpos], color='black', transform=ax.transAxes)
    line.set_clip_on(False)
    ax.add_line(line)

def label_len(my_index, level):
    labels = my_index.get_level_values(level)
    return [(k, sum(1 for _ in g)) for k, g in groupby(labels)]

def label_group_bar_table(ax, df):
    xpos = -0.2
    scale = 1. / df.index.size
    for level in range(df.index.nlevels):
        pos = df.index.size
        for label, rpos in label_len(df.index, level):
            add_line(ax, pos * scale, xpos)
            pos -= rpos
            lypos = (pos + 0.5 * rpos) * scale
            ax.text(xpos + 0.1, lypos, label, ha='center', transform=ax.transAxes)
        add_line(ax, pos * scale, xpos)
        xpos -= 0.2

# --- Crea pivot table con MultiIndex per heatmap ---
df_pivot = df.pivot_table(index=['role', 'rank'], columns='nazionality', values='counter', aggfunc='sum').fillna(0)
df_pivot = df_pivot[df_pivot.sum(axis=0).sort_values(ascending=False).index]

# --- Visualizzazione con heatmap e raggruppamento etichette ---
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111)
sns.heatmap(df_pivot, ax=ax, annot=True, fmt=".0f", cmap='YlOrBr', linewidths=1, linecolor='gray')
# Rimuove le etichette automatiche sull'asse Y
ax.set_yticklabels(['' for _ in ax.get_yticklabels()])
ax.set_ylabel('')

# Aggiunge le etichette raggruppate
label_group_bar_table(ax, df_pivot)

fig.subplots_adjust(bottom=0.1 * df_pivot.index.nlevels)
plt.title('Distribuzione delle frequenze dei giocatori per ruolo, rank e nazionalità', fontsize=14)

buf = io.BytesIO()
plt.savefig(buf, format='png', bbox_inches='tight')
buf.seek(0)
img_base64 = base64.b64encode(buf.read()).decode('utf-8')
buf.close()
plt.close(fig)

# Mostra immagine centrata
display(HTML(f"""
<div style="display: flex; justify-content: center;">
    <img src="data:image/png;base64,{img_base64}"/>
</div>
"""))

# 3. Con quali trofei si hanno più possibilità di vincere il premio?

In [ ]:
display(Markdown("Visualizziamo un barplot che confronta la posizione nel podio con la competizione"))

# Carica il dataset
df = pd.read_csv('../Dataset/dfTrophiesBarPlot.csv')

# Definizione dei periodi
periods = ['pre-1994', '1995–2009', '2010–2015', 'post-2016']

# Titoli leggibili per il menu
period_titles = {
    'pre-1994': 'Prima del 1995',
    '1995–2009': '1995–2009',
    '2010–2015': '2010–2015',
    'post-2016': 'Dal 2016 ad oggi'
}

# Colori personalizzati
rank_colors = {
    '1st': '#FFD700',   # Oro
    '2nd': '#C0C0C0',   # Argento
    '3rd': '#CD7F32'    # Bronzo
}

# Inizializza figure
fig = go.Figure()

# Per ogni periodo, crea le tracce (una per ogni Rank)
for i, period in enumerate(periods):
    df_period = df[df['Periodo'] == period]
    
    for rank in ['1st', '2nd', '3rd']:
        df_rank = df_period[df_period['Rank'] == rank]
        fig.add_trace(go.Bar(
            x=df_rank['Competizione'],
            y=df_rank['Count'],
            name=rank,
            marker_color=rank_colors[rank],
            visible=True if i == 0 else False  # Solo il primo periodo è visibile inizialmente
        ))


# Crea i pulsanti del menu a tendina
buttons = []
for i, period in enumerate(periods):
    visibility = [False] * len(periods) * 3  # 3 Rank per periodo
    start = i * 3
    visibility[start:start+3] = [True, True, True]

    buttons.append(dict(
        label=period_titles[period],
        method="update",
        args=[
            {"visible": visibility},
            {"title.text": f"Posizionamenti per Competizione - {period_titles[period]}"}   #Aggiornamento del titolo
        ]
    ))

# Layout con menu
fig.update_layout(
    width=800,    #NON TOCCARE
    height=500,   #NON TOCCARE
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            showactive=True,
            x=1,            # Centra orizzontalmente
            #xanchor="right",
            y=1.01,            # Posiziona sul grafico
            yanchor="bottom",
            font=dict(size=10)  # Riduci la dimensione del testo del menu
        )
    ],
    barmode='group',
    title=dict(
        text=f"Posizionamenti per Competizione - {period_titles[periods[0]]}",
        x=0.5,                # Centra il titolo orizzontalmente
        xanchor='center',     # Ancoraggio centrale per precisione
        font=dict(color='black')  # (Facoltativo) colore del titolo
    ),
    xaxis=dict(
        title ='Competizione',
        showline=True,        # Mostra la linea dell'asse
        linecolor='black',    # Colore della linea dell'asse
        ticks='outside',      # Direzione esterna per i tick
        ticklen=5,            # Lunghezza dei tick in pixel
        tickwidth=1,          # Spessore dei tick
        tickcolor='black',    # Colore dei tick
        color='black',        # Colore del testo delle etichette
        tickfont=dict(color='black')
    ),
    yaxis=dict(
        title='Numero di volte sul podio',
        showline=True,
        linecolor='black',
        ticks='outside',
        ticklen=5,
        tickwidth=1,
        tickcolor='black',
        color='black',
        tickfont=dict(color='black'),
        tickformat='.0f',
        dtick=1,
        showgrid=True,           # Mostra linee orizzontali della griglia
        gridcolor='lightgray'
    ),
    plot_bgcolor='white'
)

#Mostra grafici con configurazione personalizzata della toolbar
config = {
    'modeBarButtonsToRemove': [
        'zoomOut2d', 'autoScale2d', 'toggleSpikelines',
        'hoverCompareCartesian', 'pan2d', 'zoomIn2d',
        'select2d', 'lasso2d', 'sendDataToCloud', 'toImage'
    ],
    'displaylogo': False  # Rimuove il logo di Plotly
}
#Mostra grafici
html_graph = fig.to_html(full_html=False, config=config) # Genera il codice HTML interattivo del grafico
display(HTML(html_graph))

# Conclusioni
### Quali sono i risultati delle analisi?

In [ ]:
!jupyter nbconvert Slide.ipynb --to slides \
--TagRemovePreprocessor.enabled=True \
--TagRemovePreprocessor.remove_input_tags='{"hide_input"}' \
--SlidesExporter.reveal_number='c/t' \
--SlidesExporter.reveal_scroll=False
